# Compare simulations

Compare the "toy model" from matrix_values.ipynb with the actual signal event by event, station by station

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tables as tb
from load_detector_response import load_detector_data
from time_templates.utilities.plot import plot_profile_1d
from scipy.optimize import curve_fit
import uproot4
from uncertainties import unumpy, ufloat
import json
import pandas as pd
plt.rcParams.update({"text.usetex": True, "font.size": 20, "font.family":"sans"})
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
markers = ['s', 'o', '^', 'v', 'x', '*']
lss = ['-', '--', ':', '-.']

In [ ]:
def brk_pwl(x, A, alpha, beta, gamma, c1, c2):
    """
    Broken power law with 3 indices
    Gives response per particle as function of kinetic energy
    
    """
    S = np.where(x < c2,
                 np.where(x < c1, x**alpha, c1**(alpha-beta)*x**beta),
                 c1**(alpha-beta)*c2**(beta-gamma)*x**gamma)
    return A*S


def f_cos_chi(r, theta, psi, zmax=None):
    if zmax is None:
        zmax = 7400/np.cos(theta) * np.log(880/(600*np.cos(theta)))
    Delta = r*np.tan(theta)*np.cos(psi)
    return zmax * np.cos(theta)/np.sqrt(r**2 + (zmax-Delta)**2)

def f_chi(r, theta, psi, zmax=None):
    return np.arccos(f_cos_chi(r, theta, psi, zmax))

def f_Awcd(chi, R=1.8, H=1.2):
    return (np.pi*R**2*np.abs(np.cos(chi)) + 2*R*H*np.sin(chi))

def f_Assd(chi):
    return 3.84*np.abs(np.cos(chi))

def f_Lwcd(chi, R=1.8, H=1.2):
    return 1/(np.abs(np.cos(chi)) + 2*H/(np.pi*R)*np.sin(chi))

def f_Lssd(chi):
    return 1/np.cos(chi)


with open('../data/UUB_energy_response_parameters.json', 'r') as infile:
    energy_response_parameters = json.load(infile)

with open('../data/UUB_muon_response_pdf.json', 'r') as infile:
    muon_response_pdf = json.load(infile)


Rmuwcd = lambda x: brk_pwl(10**x/1e6, *energy_response_parameters['muon_wcd'])
Rmussd = lambda x: brk_pwl(10**x/1e6, *energy_response_parameters['muon_ssd'])
Relwcd = lambda x: brk_pwl(10**x/1e6, *energy_response_parameters['electron_wcd'])
Relssd = lambda x: brk_pwl(10**x/1e6, *energy_response_parameters['electron_ssd'])
Rphwcd = lambda x: brk_pwl(10**x/1e6, *energy_response_parameters['photon_wcd'])
Rphssd = lambda x: brk_pwl(10**x/1e6, *energy_response_parameters['photon_ssd'])

In [ ]:
def Lwcd_lgE_dependence(lgE, chi, EVEM=300e6):
    Lw = f_Lwcd(chi)
    lgEVEM = np.log10(EVEM)
    return Lw*np.heaviside(lgE - lgEVEM, 0)*Rmuwcd(lgE) + np.heaviside(lgEVEM - lgE, 0)*Rmuwcd(lgE)

In [ ]:
from scipy.integrate import quad
def integrate_spec(f, low=4, high=14, step=None, args=()):
#     return quad(f, low, high)[0]
    if step is None:
        step = (high - low)/10000
    x = np.arange(low, high, step)
    return f(x, *args).sum()*step

def calc_wcd_em_signal(spec_el, spec_ph, chi=0):
    Aw = f_Awcd(chi)
    Sel = integrate_spec(lambda x: spec_el(x)*Relwcd(x))
    Sph = integrate_spec(lambda x: spec_ph(x)*Rphwcd(x))
    return Aw * (Sel+Sph)

def calc_ssd_em_signal(spec_el, spec_ph, chi=0):
    As = f_Assd(chi)
    Ls = f_Lssd(chi)
    Sel = integrate_spec(lambda x: spec_el(x)*Relssd(x))
    Sph = integrate_spec(lambda x: spec_ph(x)*Rphssd(x))
    return As * Ls*(Sel+Sph)

def calc_wcd_mu_signal(spec_mu, chi=0):
    Aw = f_Awcd(chi)
    Lw = f_Lwcd(chi)
    return Aw * integrate_spec(lambda x: spec_mu(x) * Lwcd_lgE_dependence(x, chi))

def calc_ssd_mu_signal(spec_mu, chi=0):
    As = f_Assd(chi)
    Ls = f_Lssd(chi)
    return As  * Ls * integrate_spec(lambda x: spec_mu(x)*Rmussd(x))
    
def calc_average_wcd_em_signal_per_particle(spec_el, spec_ph):

    Ee = integrate_spec(lambda x: spec_el(x)*Relwcd(x))
    Ne = integrate_spec(lambda x: spec_el(x))
    
    Eph = integrate_spec(lambda x: spec_ph(x)*Rphwcd(x))
    Nph = integrate_spec(lambda x: spec_ph(x))
    
    return (Ee + Eph)/(Ne+Nph)

def calc_average_ssd_em_signal_per_particle(spec_el, spec_ph):

    Ee = integrate_spec(lambda x: spec_el(x)*Relssd(x))
    Ne = integrate_spec(lambda x: spec_el(x))
    
    Eph = integrate_spec(lambda x: spec_ph(x)*Rphssd(x))
    Nph = integrate_spec(lambda x: spec_ph(x))
    
    return (Ee + Eph)/(Ne+Nph)

def calc_average_wcd_mu_signal_per_particle(spec_mu):

    Emu = integrate_spec(lambda x: spec_mu(x)*Rmuwcd(x))
    Nmu = integrate_spec(lambda x: spec_mu(x))
    
    return Emu/Nmu

def calc_average_ssd_mu_signal_per_particle(spec_mu):

    Emu = integrate_spec(lambda x: spec_mu(x)*Rmussd(x))
    Nmu = integrate_spec(lambda x: spec_mu(x))
    
    return Emu/Nmu

In [ ]:
theta = 0
lgE = 19

directory = f'/home/mart/auger/data/CorSims_aab/Espec_hists/proton/'#'{lgE}/Z{theta}/'

import os
import uproot

rootfiles = []
for subdir, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.root'):
            try:
                hist = uproot.open(os.path.join(subdir, file))
            except OSError:
                continue
            rootfiles.append(os.path.join(subdir, file))

from re import findall
def split_key(key):
    "Get info from the tree name for file reading root, regex magic"

    try:
        lgE, ct, Xmax, _ = findall(r"[-+]?\d*\.\d+|\d+", key)
    except ValueError:
        print(key)
    if 'electron' in key:
        ptype = 'electron'
    elif 'photon' in key:
        ptype = 'photon'
    else:
        ptype = 'muon'
    return float(lgE), float(ct), float(Xmax), ptype


In [ ]:


# hist = uproot.open('/home/mart/auger/data/CorSims_aab/Espec_hists/proton/19/Z0/EPOS-fluka_p_19_0_2_DAT000002_Espec_hists.root')

hist = uproot.open('/home/mart/auger/data/CorSims_aab/Espec_hists/new/EPOS-fluka_p_19_0_2_DAT000002_Espec_hists.root')
ir = 10

f, ax = plt.subplots(1, figsize=(8, 6))

colors = ['y', 'r', 'b', 'g', 'm', 'k']

for key, c in zip(hist.keys(), colors):
    spec, lgr_bins, psi_bins, lgE_pkin_bins = hist[key].to_numpy(flow=True)
    
    lgE_pkin_bins[0] = np.log10(0.05e6)
    lgE_pkin_bins[-1] = 13    
    lgE = (lgE_pkin_bins[1:] + lgE_pkin_bins[:-1])/2

    ax.plot(lgE, spec[ir].sum(axis=0), color=c, drawstyle='steps-mid')

ax.set_yscale('log')

from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='y'),
                Line2D([0], [0], color='r'),
                Line2D([0], [0], color='b'),
                Line2D([0], [0], color='g'),
                Line2D([0], [0], color='m'),
                Line2D([0], [0], color='k')]

ax.legend(custom_lines, ['photons', 'electrons', 'muons', 'protons', 'neutrons', 'other'], fontsize=18)

ax.set_xlabel('$\\log_{10}(E_{\\rm kin}/\\mathrm{eV})$')
ax.set_ylabel('$\\frac{dN}{d\\lg{\\left(E/\\mathrm{eV}\\right)}}$')

ax.set_title(f'${10**lgr_bins[ir]:.0f} < r/\\mathrm{{m}} < {10**lgr_bins[ir+1]:.0f}$', fontsize=18)
ax.set_ylim([1e2, None])
ax.set_xlim([7, 10.5])
plt.tight_layout()

plt.savefig("Espec_EPOS_p_lgE19_Z0_r1000.pdf", bbox_inches='tight')

In [ ]:
hist = uproot.open('/home/mart/auger/data/CorSims_aab/Espec_hists/proton/19/Z0/EPOS-fluka_p_19_0_2_DAT000002_Espec_hists.root')


f, (ax, ax2) = plt.subplots(1, 2, figsize=(15, 6))

colors = ['y', 'r', 'b', 'g']
labels = ['photons', 'electrons', 'muons']
for ir, ls in zip([5, 10, 15], ['-', '--', ':']):

    for key, c in zip(hist.keys(), colors):
        spec, lgr_bins, psi_bins, lgE_pkin_bins = hist[key].to_numpy(flow=True)

        lgE_pkin_bins[0] = np.log10(0.05e6)
        lgE_pkin_bins[-1] = 13    
        lgE = (lgE_pkin_bins[1:] + lgE_pkin_bins[:-1])/2
        if 'hadron' in key:
            continue
        spec = spec[ir].sum(axis=0)
        ax.plot(lgE, spec/spec.sum(), color=c, drawstyle='steps-mid', ls=ls)
    labels.append(f'$ {10**lgr_bins[ir]:.0f} < r/\\mathrm{{m}} < {10**lgr_bins[ir+1]:.0f}$')

ax.set_yscale('log')
ax2.set_yscale('log')
custom_lines = [Line2D([0], [0], color='y'),
                Line2D([0], [0], color='r'),
                Line2D([0], [0], color='b'),
                Line2D([0], [0], color='k', ls='-'),
                Line2D([0], [0], color='k', ls='--'),
                Line2D([0], [0], color='k', ls=':')]

ax.legend(custom_lines, labels, fontsize=16, ncol=2)
ax.set_xlabel('$\\log_{10}(E_{\\rm kin}/\\mathrm{eV})$')
ax.set_ylabel('$\\frac{1}{N}\\frac{dN}{d\\lg{\\left(E/\\mathrm{eV}\\right)}}$')

ax.set_xlim([4.5, 10.5])
ax.set_ylim([1e-5, None])

labels = []

for theta, ls in zip([0, 30, 45], ['-', '--', ':']):
    hist = uproot.open(f'/home/mart/auger/data/CorSims_aab/Espec_hists/proton/19/Z{theta}/EPOS-fluka_p_19_{theta}_2_DAT000002_Espec_hists.root')
    for key, c in zip(hist.keys(), colors):
        spec, lgr_bins, psi_bins, lgE_pkin_bins = hist[key].to_numpy(flow=True)

        lgE_pkin_bins[0] = np.log10(0.05e6)
        lgE_pkin_bins[-1] = 13    
        lgE = (lgE_pkin_bins[1:] + lgE_pkin_bins[:-1])/2
        if 'hadron' in key:
            continue
        spec = spec[10].sum(axis=0)
        ax2.plot(lgE, spec/spec.sum(), color=c, drawstyle='steps-mid', ls=ls)

    labels.append(f'$\\theta = {theta:.0f}^\\circ$')
    
custom_lines = [Line2D([0], [0], color='k', ls='-'),
                Line2D([0], [0], color='k', ls='--'),
                Line2D([0], [0], color='k', ls=':')]
ax2.legend(custom_lines, labels, fontsize=16)
ax2.set_xlabel('$\\log_{10}(E_{\\rm kin}/\\mathrm{eV})$')
ax2.set_ylabel('$\\frac{1}{N}\\frac{dN}{d\\lg{\\left(E/\\mathrm{eV}\\right)}}$')
ax2.set_ylim([1e-5, None])
ax2.set_xlim([4.5, 10.5])
plt.tight_layout()

plt.savefig("Espec_EPOS_p_lgE19_theta_r_dep.pdf", bbox_inches='tight')

In [ ]:
from collections import defaultdict
from scipy.interpolate import interp1d
from scipy import integrate as scint

def load_spec(fl, dd=None, Ech=40e6, Evem=400e6):
    
    if Ech == 0:
        lgEch = -1
    else:
        lgEch = np.log10(Ech)
    lgEvem = np.log10(Evem)
    if dd is None:
        dd = defaultdict(list)
    
    try:
        hist = uproot.open(fl)
    except OSError:
        return
    
#     dd = defaultdict(list)
    for key in hist.iterkeys(filter_name='muon*'): #just getting the key name (no loop actually)
        lgE, cosTheta, Xmax, ptype = split_key(key)
        muon_spec, lgr_bins, psi_bins, lgE_pkin_bins = hist[key].to_numpy(flow=True)
        lgr_bins[0] = 2
        lgr_bins[-1] = 4
        lgE_pkin_bins[0] = np.log10(0.05e6) #from ECUTS corsika Aab
        lgE_pkin_bins[-1] = 14
        lgEmean = (lgE_pkin_bins[1:] + lgE_pkin_bins[:-1])/2
        Emean = 10**lgEmean
        lgrmean = (lgr_bins[1:] + lgr_bins[:-1])/2
        
        dlgE = lgE_pkin_bins[1:] - lgE_pkin_bins[:-1]
        electron_key = 'electron_' + '_'.join(key.split('_')[1:])
        electron_spec = hist[electron_key].values(flow=True)
        photon_key = 'photon_' + '_'.join(key.split('_')[1:])
        photon_spec = hist[photon_key].values(flow=True)
        
        #ignoring psi dependence
        mu_spec = muon_spec.sum(axis=1)#/dlgE
        el_spec = electron_spec.sum(axis=1)#/dlgE
        ph_spec = photon_spec.sum(axis=1)#/dlgE
        
        for i in range(1, len(lgrmean)-1):
            lgr = lgrmean[i]
            lgrl = lgr_bins[i]
            lgrh = lgr_bins[i+1]
            mu_spec_r = interp1d(lgEmean, mu_spec[i]/dlgE, bounds_error=False, fill_value=0)
            mu_sec_r = np.interp
            ry = ph_spec[i].sum()/el_spec[i].sum()
            aveEe = (el_spec[i] * Emean).sum()/(el_spec[i]).sum()
            aveEph = (ph_spec[i] * Emean).sum()/(ph_spec[i]).sum()
            
            mu_spec_E2 = lambda x: mu_spec_r(x)*10**(2*x)
            Nmu = mu_spec[i].sum()

            Emu1_sq = integrate_spec(mu_spec_E2, lgEch, lgEvem) / \
                Nmu

            fmu2 = integrate_spec(mu_spec_r, lgEvem, 14) / Nmu
            dd['lgE'].append(lgE)
            dd['cosTheta'].append(cosTheta)
            dd['Xmax'].append(Xmax)
            dd['lgr'].append(lgr)
            dd['lgrl'].append(lgrl)
            dd['lgrh'].append(lgrh)
            dd['filename'].append(file)

            dd['ry'].append(ry)
            dd['Ee'].append(aveEe)
            dd['Ey'].append(aveEph)
            dd['Emu1sq'].append(Emu1_sq/Evem**2)
            dd['fmu2'].append(fmu2)
            dd['theta'].append(np.rad2deg(np.arccos(cosTheta)))
            

In [ ]:
directory = '/home/mart/auger/data/time_templates/Espec_roothists/lgr/EPOS_LHC/proton/19_19.5/'

dd = defaultdict(list)

maxfiles = 1000
n = 0

for fl in os.listdir(directory):
    n += 1
    print(f'\r{n}/{maxfiles}', end='')
    load_spec(os.path.join(directory, fl), dd)
    if n > maxfiles:
        break

In [ ]:
len(os.listdir(directory))

In [ ]:
dd

In [ ]:
from collections import defaultdict
from scipy.interpolate import interp1d

dd = defaultdict(list)

hists = uproot.open(rootfiles[0])

h, lgr_bins, psi_bins, lgE_pkin_bins = hists[hists.keys()[0]].to_numpy(flow=True)

lgr_bins[0] = 2
lgr_bins[-1] = 4
lgE_pkin_bins[0] = np.log10(0.05e6) #from ECUTS corsika Aab
lgE_pkin_bins[-1] = 14
lgEmean = (lgE_pkin_bins[1:] + lgE_pkin_bins[:-1])/2
lgrmean = (lgr_bins[1:] + lgr_bins[:-1])/2
dlgE = lgE_pkin_bins[1:] - lgE_pkin_bins[:-1]

for file in rootfiles:
    hist = uproot.open(file)
    for key in hist.iterkeys(filter_name='muon*'):
        lgE, cosTheta, Xmax, ptype = split_key(key)
        muon_spec = hist[key].values(flow=True)
        electron_key = 'electron_' + '_'.join(key.split('_')[1:])
        electron_spec = hist[electron_key].values(flow=True)
        photon_key = 'photon_' + '_'.join(key.split('_')[1:])
        photon_spec = hist[photon_key].values(flow=True)
#         hadron_key = 'hadron_' + '_'.join(key.split('_')[1:])
#         hadron_spec = hist[hadron_key].values(flow=True)
        
        mu_spec = muon_spec.sum(axis=1)/dlgE
        el_spec = electron_spec.sum(axis=1)/dlgE
        ph_spec = photon_spec.sum(axis=1)/dlgE
#         hd_spec = hadron_spec.sum(axis=1)/dlgE
        
        for i in range(1, len(lgrmean)-1):
            lgr = lgrmean[i]
            lgrl = lgr_bins[i]
            lgrh = lgr_bins[i+1]
            Asampling = np.pi*(10**(2*lgr_bins[i+1]) - 10**(2*lgr_bins[i]))/cosTheta
            
            mu_spec_r = interp1d(lgEmean, mu_spec[i]/Asampling, fill_value=0, bounds_error=False)
#             el_spec_r = interp1d(lgEmean, el_spec[i]/Asampling, fill_value=0, bounds_error=False)
#             ph_spec_r = interp1d(lgEmean, ph_spec[i]/Asampling, fill_value=0, bounds_error=False)
            
            ry = ph_spec[i].sum()/el_spec[i].sum()
            aveEe = (el_spec[i]*dlgE * 10**lgEmean).sum()/(el_spec[i]*dlgE).sum()
            aveEph = (ph_spec[i]*dlgE * 10**lgEmean).sum()/(ph_spec[i]*dlgE).sum()
#             aveEe = integrate_spec(lambda x: el_spec_r(x)*10**x)/integrate_spec(el_spec_r)
#             aveEph = integrate_spec(lambda x: ph_spec_r(x)*10**x)/integrate_spec(ph_spec_r)
            Evem = 400e6
            Ech = 40e6
            Emu1_sq = integrate_spec(lambda x: mu_spec_r(x)*(10**x)**2, np.log10(Ech), np.log10(Evem)) / \
                integrate_spec(mu_spec_r)
            fmu2 = integrate_spec(mu_spec_r, np.log10(Evem)) / integrate_spec(mu_spec_r)

#             chi = np.arccos(cosTheta) #f_chi(10**lgr, np.arccos(cosTheta), np.pi/2, zmax=1000)
#             Swcd_em = calc_wcd_em_signal(el_spec_r, ph_spec_r, chi)
#             Sssd_em = calc_ssd_em_signal(el_spec_r, ph_spec_r, chi)   
#             Swcd_mu = calc_wcd_mu_signal(mu_spec_r, chi)
#             Sssd_mu = calc_ssd_mu_signal(mu_spec_r, chi)

#             b = calc_average_wcd_mu_signal_per_particle(mu_spec_r)
#             d = calc_average_ssd_mu_signal_per_particle(mu_spec_r)
#             a = calc_average_wcd_em_signal_per_particle(el_spec_r, ph_spec_r)
#             c = calc_average_ssd_em_signal_per_particle(el_spec_r, ph_spec_r)

            dd['lgE'].append(lgE)
            dd['cosTheta'].append(cosTheta)
            dd['Xmax'].append(Xmax)
            dd['lgr'].append(lgr)
            dd['lgrl'].append(lgrl)
            dd['lgrh'].append(lgrh)
            dd['filename'].append(file)
#             dd['Swcd_em'].append(Swcd_em)
#             dd['Sssd_em'].append(Sssd_em)
#             dd['Swcd_mu'].append(Swcd_mu)
#             dd['Sssd_mu'].append(Sssd_mu)
#             dd['alpha'].append(Sssd_em/Swcd_em)
#             dd['beta'].append(Sssd_mu/Swcd_mu)
#             dd['a'].append(a)
#             dd['b'].append(b)
#             dd['c'].append(c)
#             dd['d'].append(d)
            dd['ry'].append(ry)
            dd['Ee'].append(aveEe)
            dd['Ey'].append(aveEph)
            dd['Emu1sq'].append(Emu1_sq)
            dd['fmu2'].append(fmu2)
            dd['theta'].append(np.rad2deg(np.arccos(cosTheta)))

df = pd.DataFrame(dd)

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15, 6))

df_19_0 = df.query('theta == 0 & lgE == 19.0')


colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, file in enumerate(df_19_0['filename'].unique()):
    
    df_ = df_19_0.loc[df_19_0['filename'] == file]

    axes[0].hlines(df_['ry'], df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)
    axes[1].hlines(df_['Ee']/1e6, df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)
    axes[2].hlines(df_['Ey']/1e6, df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)

axes[0].set_ylabel('$r_\\gamma$')
axes[1].set_ylabel('$\\langle E_e \\rangle$ [MeV]')
axes[2].set_ylabel('$\\langle E_\\gamma \\rangle$ [MeV]')

df_mean = df_19_0.groupby(['lgr']).mean().reset_index()
df_std = df_19_0.groupby(['lgr']).std().reset_index()

axes[0].errorbar(df_mean['lgr'], df_mean['ry'], xerr=0.02, yerr=df_std['ry']/np.sqrt(10),
                 ls='', color='k', lw=3)
axes[1].errorbar(df_mean['lgr'], df_mean['Ee']/1e6, xerr=0.02, yerr=df_std['Ee']/np.sqrt(10)/1e6,
                 ls='', color='k', lw=3)
axes[2].errorbar(df_mean['lgr'], df_mean['Ey']/1e6, xerr=0.02, yerr=df_std['Ey']/np.sqrt(10)/1e6,
                 ls='', color='k', lw=3)
for ax in axes:
    ax.set_xlabel('$\\log_{10}(r/\mathrm{m})$')
plt.tight_layout()

plt.savefig('ry_Ee_Ey_vs_r.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11, 6))

Evem = 400e6

df_19_0 = df.query('theta == 0 & lgE == 19.0')

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, file in enumerate(df_19_0['filename'].unique()):
    df_ = df_19_0.loc[df_19_0['filename'] == file]

    axes[0].hlines(df_['Emu1sq']/(Evem)**2, df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)
    axes[1].hlines(df_['fmu2'], df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)


df_mean = df_19_0.groupby(['lgr']).mean().reset_index()
df_std = df_19_0.groupby(['lgr']).std().reset_index()

axes[0].errorbar(df_mean['lgr'], df_mean['Emu1sq']/(Evem)**2, xerr=0.02,
                 yerr=df_std['Emu1sq']/np.sqrt(10)/(Evem)**2,
                 ls='', color='k', lw=3)
axes[1].errorbar(df_mean['lgr'], df_mean['fmu2'], xerr=0.02, yerr=df_std['fmu2']/np.sqrt(10),
                 ls='', color='k', lw=3)
axes[0].set_ylabel('$\\langle  \\left(E^\\mu_1\\right)^2 \\rangle / E_{\\rm VEM}^2$')
axes[1].set_ylabel('$f^\mu_2$')
for ax in axes:
    ax.set_xlabel('$\\log_{10}(r/\mathrm{m})$')

plt.tight_layout()
plt.savefig('Emu21_fmu2_vs_r.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15, 6))

df_19_r1000 = df.query('lgrh == 3 & lgE == 19.0')

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, Xmax in enumerate(df_19_r1000['Xmax'].unique()):
    df_ = df_19_r1000.loc[df_19_r1000['Xmax'] == Xmax]
    axes[0].plot(df_['theta'], df_['ry'], marker='o', ls='', ms=4)
    axes[1].plot(df_['theta'], df_['Ee']/1e6, marker='o', ls='', ms=4)
    axes[2].plot(df_['theta'], df_['Ey']/1e6, marker='o', ls='', ms=4)
    
axes[0].set_ylabel('$r_\\gamma$')
axes[1].set_ylabel('$\\langle E_e \\rangle$ [MeV]')
axes[2].set_ylabel('$\\langle E_\\gamma \\rangle$ [MeV]')

df_mean = df_19_r1000.groupby(['theta']).mean().reset_index()
df_std = df_19_r1000.groupby(['theta']).std().reset_index()

axes[0].errorbar(df_mean['theta'], df_mean['ry'], yerr=df_std['ry']/np.sqrt(10),
                 ls='', color='k', zorder=99, marker='o', ms=10, lw=3)
axes[1].errorbar(df_mean['theta'], df_mean['Ee']/1e6,yerr=df_std['Ee']/np.sqrt(10)/1e6,
                  ls='', color='k', zorder=99, marker='o', ms=10, lw=3)
axes[2].errorbar(df_mean['theta'], df_mean['Ey']/1e6, yerr=df_std['Ey']/np.sqrt(10)/1e6,
                 ls='', color='k', zorder=99, marker='o', ms=10, lw=3)
for ax in axes:
    ax.set_xlabel('$\\theta$ [deg]')
plt.tight_layout()
plt.savefig('ry_Ee_Ey_vs_theta.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11, 6))

df_19_r1000 = df.query('lgrh == 3 & lgE == 19.0')

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, Xmax in enumerate(df_19_r1000['Xmax'].unique()):
    df_ = df_19_r1000.loc[df_19_r1000['Xmax'] == Xmax]
    axes[0].plot(df_['theta'], df_['Emu1sq']/(Evem)**2, marker='o', ls='', ms=4)
    axes[1].plot(df_['theta'], df_['fmu2'], marker='o', ls='', ms=4)


df_mean = df_19_r1000.groupby(['theta']).mean().reset_index()
df_std = df_19_r1000.groupby(['theta']).std().reset_index()

axes[0].errorbar(df_mean['theta'], df_mean['Emu1sq']/(Evem)**2, yerr=df_std['Emu1sq']/np.sqrt(10)/(Evem)**2,
                 ls='', color='k', zorder=99, marker='o', ms=10, lw=4)
axes[1].errorbar(df_mean['theta'], df_mean['fmu2'], yerr=df_std['fmu2']/np.sqrt(10),
                  ls='', color='k', zorder=99, marker='o', ms=10, lw=4)
axes[0].set_ylabel('$\\langle  \\left(E^\\mu_1\\right)^2 \\rangle / E_{\\rm VEM}^2$')
axes[1].set_ylabel('$f^\mu_2$')
for ax in axes:
    ax.set_xlabel('$\\theta$ [deg]')
plt.tight_layout()
plt.savefig('Emu21_fmu2_vs_theta.pdf', bbox_inches='tight')

In [ ]:
ry = 25
Ee = 27e6 
Ey = 7e6

Emu12 = 1.9e4 *(1e6)**2 #MeV^2
fmu2 = 0.65

eps_y_ssd = 0.03
eps_e_ssd = 1

G = 0.378
Evem = 400e6

ssdfactor = 1.13
ssd_mu_factor = 1
Lw = 1

alpha = ssdfactor * G * Evem * (eps_e_ssd + eps_y_ssd*ry)/(Ee + ry*Ey)
beta = ssdfactor * G  * ssd_mu_factor / (Emu12/Evem**2 + fmu2*Lw)
print(alpha, beta)

df['alpha'] = ssdfactor * G * Evem * (eps_e_ssd + eps_y_ssd*df['ry'])/(df['Ee'] + df['ry']*df['Ey'])
df['beta'] = ssdfactor * G * ssd_mu_factor / (df['Emu1sq']/Evem**2 + df['fmu2'])

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))

df_19_0 = df.query('theta == 0 & lgE == 19.0')

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, file in enumerate(df_19_0['filename'].unique()):
    df_ = df_19_0.loc[df_19_0['filename'] == file]

    ax1.hlines(df_['alpha'], df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)
    ax2.hlines(df_['beta'], df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)



df_mean = df_19_0.groupby(['lgr']).mean().reset_index()
df_std = df_19_0.groupby(['lgr']).std().reset_index()

ax1.errorbar(df_mean['lgr'], df_mean['alpha'], xerr=0.02, yerr=df_std['alpha']/np.sqrt(10),
                 ls='', color='k', lw=3)
ax2.errorbar(df_mean['lgr'], df_mean['beta'], xerr=0.02, yerr=df_std['beta']/np.sqrt(10),
                 ls='', color='k', lw=3)


ax1.set_xlabel('$\\log_{10}(r/\mathrm{m})$')
ax2.set_xlabel('$\\log_{10}(r/\mathrm{m})$')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()
plt.savefig('alpha_beta_vs_r_toy.pdf', bbox_inches='tight')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))

df_19_r1000 = df.query('lgrh == 3 & lgE == 19.0')

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, Xmax in enumerate(df_19_r1000['Xmax'].unique()):
    df_ = df_19_r1000.loc[df_19_r1000['Xmax'] == Xmax]
    ax1.plot(df_['theta'], df_['alpha'], marker='o', ls='', ms=4)
    ax2.plot(df_['theta'], df_['beta'], marker='o', ls='', ms=4)


df_mean = df_19_r1000.groupby(['theta']).mean().reset_index()
df_std = df_19_r1000.groupby(['theta']).std().reset_index()

ax1.errorbar(df_mean['theta'], df_mean['alpha'], yerr=df_std['alpha']/np.sqrt(10),
                 ls='', color='k', zorder=99, marker='o', ms=10, lw=4)
ax2.errorbar(df_mean['theta'], df_mean['beta'], yerr=df_std['beta']/np.sqrt(10),
                  ls='', color='k', zorder=99, marker='o', ms=10, lw=4)


plt.tight_layout()
ax1.set_xlabel('$\\theta$ [deg]')
ax2.set_xlabel('$\\theta$ [deg]')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()
plt.savefig('alpha_beta_vs_theta_toy.pdf', bbox_inches='tight')

In [ ]:
df_rec = pd.read_pickle('../data/df_Aab_UUB_SSD_EPOS_p_dense.pl')
# df_rec = df_rec.query('LowGainSat == 0')
df_rec['MCr'] = df_rec['MCr'].apply(round)
df_rec['MCTheta'] = df_rec['MCTheta'].apply(lambda x: np.round(x, 4))

In [ ]:
def get_uncertainty_pandas(x, n):
    s1 = np.sum(x)
    s2 = np.sum(x**2)
    
    std = np.sqrt(s2/n - (s1/n)**2)
    return std/np.sqrt(n)

def calc_uncertainty_ab(wcd, ssd, wcd_err, ssd_err):
    ab = ssd/wcd
    corr = 1
    return ab * np.sqrt(wcd_err**2/wcd**2 + ssd_err**2/ssd**2 - corr*wcd_err/wcd * ssd_err/ssd)


def make_alpha_beta_plot(df, cosTheta, lgE, axes=None, **pltkwargs):
    df_q = df.query(f'MCCosTheta > {cosTheta-0.01} & MCCosTheta < {cosTheta+0.02} & MClgE > {lgE-0.1} & MClgE < {lgE+0.1}')
    df_gb = df_q.groupby(['MCr'])
    #beware: do not get mean, but sum and divide by number of stations before trigger: (12 * nsims=10)
    df_gb_mean = df_gb.sum().reset_index()
    r = df_gb_mean['MCr'].values
    m = 9*32
    ssd_em_signal = df_gb_mean['SSDEMSignal'].values/m
    wcd_em_signal = df_gb_mean['WCDEMSignal'].values/m
    ssd_muon_signal = df_gb_mean['SSDMuonSignal'].values/m
    wcd_muon_signal = df_gb_mean['WCDMuonSignal'].values/m

    ssd_em_signal_err = df_gb['SSDEMSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    wcd_em_signal_err = df_gb['WCDEMSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    ssd_muon_signal_err = df_gb['SSDMuonSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    wcd_muon_signal_err = df_gb['WCDMuonSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values

    alpha = ssd_em_signal/wcd_em_signal
    beta = ssd_muon_signal/wcd_muon_signal

    alpha_err = calc_uncertainty_ab(wcd_em_signal, ssd_em_signal, wcd_em_signal_err, ssd_em_signal_err)
    beta_err = calc_uncertainty_ab(wcd_muon_signal, ssd_muon_signal, wcd_muon_signal_err, ssd_muon_signal_err)

    if axes is None:
        f, axes = plt.subplots(1, 2, figsize=(15, 7))

    axes[0].errorbar(r, alpha, alpha_err, **pltkwargs)

    axes[1].errorbar(r, beta, beta_err, **pltkwargs)

    axes[0].set_xlabel('r [m]')
    axes[1].set_xlabel('r [m]')
    axes[0].set_ylabel('alpha')
    axes[1].set_ylabel('beta')
    return r, alpha, alpha_err, beta, beta_err

In [ ]:


f, axes = plt.subplots(1, 2, figsize=(11, 6), sharex=True)


ax1, ax2 = axes

df_19_0 = df.query('theta == 0 & lgE == 19.0')

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]


df_mean = df_19_0.groupby(['lgr']).mean().reset_index()
df_std = df_19_0.groupby(['lgr']).std().reset_index()

xerr = 10**df_mean['lgr'] *(10**0.02 - 1)
ax1.errorbar(10**df_mean['lgr'], df_mean['alpha'], xerr=xerr, yerr=df_std['alpha']/np.sqrt(10),
                 ls='', color='k', lw=2)
ax2.errorbar(10**df_mean['lgr'], df_mean['beta'], xerr=xerr, yerr=df_std['beta']/np.sqrt(10),
                 ls='', color='k', lw=2)

make_alpha_beta_plot(df_rec, np.cos(np.deg2rad(0)), 19, axes=axes, marker='s', ls='');

ax1.set_xlabel('$r\ [\\rm m]$')
ax2.set_xlabel('$r\ [\\rm m]$')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()
plt.savefig('alpha_beta_vs_r_toy_and_signal.pdf', bbox_inches='tight')


ax1.errorbar([0], [0], marker='', xerr=0.5, yerr=0.5, label='from toy model', color='k')
ax1.errorbar([0], [0], marker='s', yerr=0.5, xerr=0, label='from reconstructed signal', color='C0')
ax1.legend(fontsize=16)
ax1.set_xlim([350, 2000])
ax1.set_ylim([1.2, 1.75])
ax2.set_ylim([0.45, 0.8])

In [ ]:
def make_alpha_beta_plot_vs_theta(df, r, lgE, axes=None, **pltkwargs):
    df_q = df.query(f'MClgE > {lgE-0.1} & MClgE < {lgE+0.1} & MCr == {r}')
    df_q['MCCosTheta'] = df_q['MCCosTheta'].round(2)
    df_gb = df_q.groupby(['MCCosTheta'])
    #beware: do not get mean, but sum and divide by number of stations before trigger: (12 * nsims=10)
    df_gb_mean = df_gb.sum().reset_index()
    theta = np.arccos(df_gb_mean['MCCosTheta'].values)
    m = 9*32
    ssd_em_signal = df_gb_mean['SSDEMSignal'].values/m
    wcd_em_signal = df_gb_mean['WCDEMSignal'].values/m
    ssd_muon_signal = df_gb_mean['SSDMuonSignal'].values/m
    wcd_muon_signal = df_gb_mean['WCDMuonSignal'].values/m

    ssd_em_signal_err = df_gb['SSDEMSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    wcd_em_signal_err = df_gb['WCDEMSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    ssd_muon_signal_err = df_gb['SSDMuonSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    wcd_muon_signal_err = df_gb['WCDMuonSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values

    alpha = ssd_em_signal/wcd_em_signal
    beta = ssd_muon_signal/wcd_muon_signal

    alpha_err = calc_uncertainty_ab(wcd_em_signal, ssd_em_signal, wcd_em_signal_err, ssd_em_signal_err)
    beta_err = calc_uncertainty_ab(wcd_muon_signal, ssd_muon_signal, wcd_muon_signal_err, ssd_muon_signal_err)

    if axes is None:
        f, axes = plt.subplots(1, 2, figsize=(15, 7))

    axes[0].errorbar(np.rad2deg(theta), alpha, alpha_err, **pltkwargs)

    axes[1].errorbar(np.rad2deg(theta), beta, beta_err, **pltkwargs)

    axes[0].set_xlabel('$\\theta$ [deg]')
    axes[1].set_xlabel('$\\theta$ [deg]')
    axes[0].set_ylabel('alpha')
    axes[1].set_ylabel('beta')
    return theta, alpha, alpha_err, beta, beta_err

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11, 6), sharex=True)

ax1, ax2 = axes

make_alpha_beta_plot_vs_theta(df_rec, 1000, 19, axes=axes, ls='', marker='s');

df_19_r1000 = df.query('lgrh == 3 & lgE == 19.0')

df_mean = df_19_r1000.groupby(['theta']).mean().reset_index()
df_std = df_19_r1000.groupby(['theta']).std().reset_index()

ax1.errorbar(df_mean['theta'], df_mean['alpha'], yerr=df_std['alpha']/np.sqrt(10),
                 ls='', color='k', zorder=99, marker='o',
             label='from toy model ($912 < r/\mathrm{m} < 1000$)')
ax2.errorbar(df_mean['theta'], df_mean['beta'], yerr=df_std['beta']/np.sqrt(10),
                  ls='', color='k', zorder=99, marker='o')

ax1.errorbar([0], [0], marker='s', color='C0', ls='', label='from reconstructed signal ($r=1000\, \\rm m$)')
ax1.legend(fontsize=14)
plt.tight_layout()
ax1.set_xlabel('$\\theta$ [deg]')
ax2.set_xlabel('$\\theta$ [deg]')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}} $')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}(r=1000\, \\rm m)$')

ax1.set_ylim([1.0, 1.65])
ax2.set_ylim([0.35, 0.85])
plt.tight_layout()
plt.savefig('alpha_beta_vs_theta_toy_signal_1000m.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11, 6), sharex=True)

ax1, ax2 = axes

df_19_r1000 = df.query('lgrh == 3 & theta == 0')

df_mean = df_19_r1000.groupby(['lgE']).mean().reset_index()
df_std = df_19_r1000.groupby(['lgE']).std().reset_index()

ax1.errorbar(df_mean['lgE'], df_mean['alpha'], yerr=df_std['alpha']/np.sqrt(10),
                 ls='', color='k', zorder=99, marker='o',
             label='from toy model ($912 < r/\mathrm{m} < 1000$)')
ax2.errorbar(df_mean['lgE'], df_mean['beta'], yerr=df_std['beta']/np.sqrt(10),
                  ls='', color='k', zorder=99, marker='o')

# ax1.errorbar([0], [0], marker='s', color='C0', ls='', label='from reconstructed signal ($r=1000\, \\rm m$)')
# ax1.legend(fontsize=14)
# plt.tight_layout()
# ax1.set_xlabel('$\\theta$ [deg]')
# ax2.set_xlabel('$\\theta$ [deg]')

# ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}} $')
# ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}(r=1000\, \\rm m)$')

# ax1.set_ylim([1.0, 1.65])
# ax2.set_ylim([0.35, 0.85])
# plt.tight_layout()
# plt.savefig('alpha_beta_vs_theta_toy_signal_1000m.pdf', bbox_inches='tight')

In [ ]:
df_std['beta']/np.sqrt(10)

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(15, 7))
for theta in [0, 15, 30, 45, 60]:
    _ = make_alpha_beta_plot(df_rec, np.cos(np.deg2rad(theta)), 19, axes=axes, marker='s', ls='')
    
axes[0].set_ylim([1, 1.8])
axes[1].set_ylim([0, 1.1]);

From data it is also clear that sigma/sqrt(s) ~ 1.3 for vertical, this means that each particles creates ~1.5 MIP in the SSD. Is this because of the landau tail? definition of MIP??. Anyway, scaling by this factor fixed the SSD signal (both EM and muon). Now the only discrepancy seems to come from the WCD EM signal

In [ ]:
def get_Smu_Sem_matrix_inv(Sw, Ss, alpha, beta):
        
    Smu = 1/(alpha-beta) * (alpha*Sw - Ss)
    Sem = 1/(alpha-beta) * (Ss - beta*Sw)
#     Smu = 1/(alpha-beta) * np.where(alpha * Sw > Ss, (alpha*Sw - Ss), Sw*(alpha-1))
#     Sem = 1/(alpha-beta) * np.where(Ss > beta*Sw, Ss-beta*Sw, Sw*(1-beta))
    return Smu, Sem

Sw = 30
Ss = 30


f, axes = plt.subplots(2, 2, figsize=(15, 12))

Sem = 100
Smu = 100
Sw = Sem + Smu
alpha = 1.5
beta = 0.8

Ss = alpha*Sem + beta*Smu

alphas = np.linspace(1.2, 1.7)

Smu_pred, Sem_pred = get_Smu_Sem_matrix_inv(Sw, Ss, alphas, beta)
axes[0, 0].plot(alphas, Smu_pred/Smu)
axes[0, 1].plot(alphas, Sem_pred/Sem)

betas = np.linspace(0.5, 0.9)

Smu_pred, Sem_pred = get_Smu_Sem_matrix_inv(Sw, Ss, alpha, betas)
axes[1, 0].plot(betas, Smu_pred/Smu)
axes[1, 1].plot(betas, Sem_pred/Sem)

for ax in axes.flatten():
    ax.axhline(1, ls='--', color='k')
    
axes[0, 0].set_ylabel('Smupred/Smu')
axes[0, 0].set_xlabel('$\\alpha$')

axes[0, 1].set_ylabel('Sempred/Sem')
axes[0, 1].set_xlabel('$\\alpha$')

axes[1, 0].set_ylabel('Smupred/Smu')
axes[1, 0].set_xlabel('$\\beta$')

axes[1, 1].set_ylabel('Sempred/Sem')
axes[1, 1].set_xlabel('$\\beta$')

plt.tight_layout()